# Crime Analytics in Seattle and San Francisco

In [119]:
YEAR = 2014
PERIOD = "summer"
TYPE = "incidents"
DATA_DIR = "data"
SF = "sanfrancisco"
SEATTLE = "seattle"

In [120]:
# Load data into pandas DataFrames
import os
import pandas as pd

sf_filename = os.path.join(DATA_DIR, "%s_%s_%s_%d.csv" % (SF, TYPE, PERIOD, YEAR))
sf = pd.read_csv(sf_filename)
seattle_filename = os.path.join(DATA_DIR, "%s_%s_%s_%d.csv" % (SEATTLE, TYPE, PERIOD, YEAR))
seattle = pd.read_csv(seattle_filename)
assert sf.shape[0] == len(open(sf_filename).readlines()) - 1 != 0
assert seattle.shape[0] == len(open(seattle_filename).readlines()) - 1 != 0

In [121]:
sf.head(5)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,140734311,ARSON,ARSON OF A VEHICLE,Sunday,08/31/2014,23:50,BAYVIEW,NONE,LOOMIS ST / INDUSTRIAL ST,-122.405647,37.738322,"(37.7383221869053, -122.405646994567)",14073431126031
1,140736317,NON-CRIMINAL,LOST PROPERTY,Sunday,08/31/2014,23:45,MISSION,NONE,400 Block of CASTRO ST,-122.435012,37.761768,"(37.7617677182954, -122.435012093789)",14073631771000
2,146177923,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/31/2014,23:30,SOUTHERN,NONE,1000 Block of MISSION ST,-122.409795,37.780036,"(37.7800356268394, -122.409795194505)",14617792306244
3,146177531,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,08/31/2014,23:30,RICHMOND,NONE,FULTON ST / 26TH AV,-122.485263,37.772518,"(37.7725176473142, -122.485262988324)",14617753106244
4,140734220,NON-CRIMINAL,FOUND PROPERTY,Sunday,08/31/2014,23:23,RICHMOND,NONE,800 Block of LA PLAYA ST,-122.509895,37.772313,"(37.7723131976814, -122.509895418239)",14073422072000


In [122]:
seattle.head(5)

,RMS CDW ID,General Offense Number,Offense Code,Offense Code Extension,Offense Type,Summary Offense Code,Summarized Offense Description,Date Reported,Occurred Date or Date Range Start,Occurred Date Range End,Hundred Block Location,District/Sector,Zone/Beat,Census Tract 2000,Longitude,Latitude,Location,Month,Year
0,483839,2015218538,2202,0,BURGLARY-FORCE-RES,2200,BURGLARY,06/28/2015 10:31:00 AM,06/28/2014 10:31:00 AM,06/28/2015 10:31:00 AM,6XX BLOCK OF NW 74 ST,J,J2,2900.3013,-122.364672,47.682524,"(47.68252427, -122.364671996)",6,2014
1,481252,2015213067,2610,0,FRAUD-IDENTITY THEFT,2600,FRAUD,06/24/2015 11:09:00 AM,06/01/2014 12:00:00 AM,06/24/2015 11:09:00 AM,23XX BLOCK OF 43 AV E,C,C2,6300.1004,-122.277080,47.639901,"(47.639900761, -122.277080248)",6,2014
2,481375,2015210301,2316,0,THEFT-MAIL,2300,MAIL THEFT,06/22/2015 09:22:00 AM,08/31/2014 09:00:00 AM,NaN,81XX BLOCK OF 11 AV SW,F,F3,11300.5013,-122.349312,47.529232,"(47.529232299, -122.349312181)",8,2014
3,481690,2015209327,2599,0,COUNTERFEIT,2500,COUNTERFEIT,06/21/2015 03:52:00 PM,06/20/2014 01:38:00 PM,NaN,6XX BLOCK OF PINE ST,M,M2,8200.1002,-122.334818,47.612368,"(47.612368448, -122.334817763)",6,2014
4,478198,2015207880,2399,3,THEFT-OTH,2300,OTHER PROPERTY,06/20/2015 11:59:00 AM,06/01/2014 11:59:00 AM,11/01/2014 12:00:00 PM,77XX BLOCK OF SUNNYSIDE AV N,J,J3,2700.2015,-122.329379,47.685960,"(47.685959879, -122.329378505)",6,2014


In [123]:
import shapefile 

ZILLOW_PREFIX = "ZillowNeighborhoods"
CA_NEIGHBORHOODS = os.path.join(DATA_DIR, "%s-CA.shp" % ZILLOW_PREFIX)
WA_NEIGHBORHOODS = os.path.join(DATA_DIR, "%s-WA.shp" % ZILLOW_PREFIX)

def point_in_poly(x,y,poly):
    #check if point is a vertex
    if (x,y) in poly: return True

    # check if point is on a boundary
    for i in range(len(poly)):
        p1 = None
        p2 = None
        if i==0:
            p1 = poly[0]
            p2 = poly[1]
        else:
            p1 = poly[i-1]
            p2 = poly[i]
        if p1[1] == p2[1] and p1[1] == y and x > min(p1[0], p2[0]) and x < max(p1[0], p2[0]):
            return True

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

ca_shp = shapefile.Reader(CA_NEIGHBORHOODS)
wa_shp = shapefile.Reader(WA_NEIGHBORHOODS)

ca_sr = ca_shp.shapeRecords()
wa_sr = wa_shp.shapeRecords()

sf_sr = [x for x in ca_sr if x.record[2] == "San Francisco"]
seattle_sr = [x for x in wa_sr if x.record[2] == "Seattle"]

UNKNOWN = "Unknown"
def sf_neighborhood(row):
    for sr in sf_sr:
        if point_in_poly(row["Longitude"], row["Latitude"], sr.shape.points):
            return sr.record[3]
    return UNKNOWN

def seattle_neighborhood(row):
    for sr in seattle_sr:
        if point_in_poly(row["Longitude"], row["Latitude"], sr.shape.points):
            return sr.record[3]
    return UNKNOWN

In [124]:
sf.rename(columns = {"X": "Longitude", "Y": "Latitude", "PdDistrict": "District", "Descript": "Description"}, 
          inplace=True)
sf["City"] = pd.Series([SF]*sf.shape[0], index=sf.index)
sf["DayOfMonth"] = sf.Date.map(lambda x: int(x.split("/")[1]))
sf["Month"] = sf.Date.map(lambda x: int(x.split("/")[0]))
sf["Year"] = sf.Date.map(lambda x: int(x.split("/")[2]))
sf["Neighborhood"] = sf.apply(sf_neighborhood, axis=1)
sf.drop(["IncidntNum", "PdId", "Location", "Resolution"], axis=1, inplace=True, errors="ignore")
# Reindex the DataFrame
sf_columns_index = ["City", "DayOfMonth", "DayOfWeek", "Month", "Year", "Time", 
                    "Longitude", "Latitude", "Address", "District", "Neighborhood", 
                    "Category", "Description"
                   ]
sf_normalized = sf[sf_columns_index]
sf_normalized.head(5)

,City,DayOfMonth,DayOfWeek,Month,Year,Time,Longitude,Latitude,Address,District,Neighborhood,Category,Description
0,sanfrancisco,31,Sunday,8,2014,23:50,-122.405647,37.738322,LOOMIS ST / INDUSTRIAL ST,BAYVIEW,Bayview,ARSON,ARSON OF A VEHICLE
1,sanfrancisco,31,Sunday,8,2014,23:45,-122.435012,37.761768,400 Block of CASTRO ST,MISSION,Castro-Upper Market,NON-CRIMINAL,LOST PROPERTY
2,sanfrancisco,31,Sunday,8,2014,23:30,-122.409795,37.780036,1000 Block of MISSION ST,SOUTHERN,South Of Market,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO
3,sanfrancisco,31,Sunday,8,2014,23:30,-122.485263,37.772518,FULTON ST / 26TH AV,RICHMOND,Unknown,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO
4,sanfrancisco,31,Sunday,8,2014,23:23,-122.509895,37.772313,800 Block of LA PLAYA ST,RICHMOND,Outer Richmond,NON-CRIMINAL,FOUND PROPERTY


In [136]:
import datetime

seattle_drop_columns = ["RMS CDW ID", 
                        "General Offense Number", 
                        "Offense Code", 
                        "Offense Code Extension", 
                        "Summary Offense Code",
                        "Census Tract 2000",
                        "Location"
                       ]
seattle.drop(seattle_drop_columns, axis=1, inplace=True, errors="ignore")
seattle.rename(columns = {"Hundred Block Location": "Address", "Offense Type": "Category", 
                          "Summarized Offense Description": "Description",
                          "District/Sector": "District"}, 
               inplace=True)
seattle["City"] = pd.Series([SEATTLE]*seattle.shape[0], index=seattle.index)
seattle["DayOfMonth"] = seattle["Occurred Date or Date Range Start"].map(lambda x: int(x.split(" ")[0].split("/")[1]))
seattle["DayOfWeek"] = seattle["Occurred Date or Date Range Start"] \
    .map(lambda x: datetime.datetime.strptime(x.split(" ")[0], "%m/%d/%Y").strftime("%A"))
seattle["Time"] = seattle["Occurred Date or Date Range Start"] \
    .map(lambda x: datetime.datetime.strptime(x[11:], "%I:%M:%S %p").strftime("%H:%M"))
seattle["Neighborhood"] = seattle.apply(seattle_neighborhood, axis=1)
# Reindex the DataFrame
seattle_columns_index = ["City", "DayOfMonth", "DayOfWeek", "Month", "Year", "Time", 
                         "Longitude", "Latitude", "Address", "District", "Neighborhood", 
                         "Category", "Description"
                        ]
seattle_normalized = seattle[seattle_columns_index]
seattle_normalized.head(5)

,City,DayOfMonth,DayOfWeek,Month,Year,Time,Longitude,Latitude,Address,District,Neighborhood,Category,Description
0,seattle,28,Saturday,6,2014,10:31,-122.364672,47.682524,6XX BLOCK OF NW 74 ST,J,Phinney Ridge,BURGLARY-FORCE-RES,BURGLARY
1,seattle,1,Sunday,6,2014,00:00,-122.277080,47.639901,23XX BLOCK OF 43 AV E,C,Madison Park,FRAUD-IDENTITY THEFT,FRAUD
2,seattle,31,Sunday,8,2014,09:00,-122.349312,47.529232,81XX BLOCK OF 11 AV SW,F,Highland Park,THEFT-MAIL,MAIL THEFT
3,seattle,20,Friday,6,2014,13:38,-122.334818,47.612368,6XX BLOCK OF PINE ST,M,Downtown,COUNTERFEIT,COUNTERFEIT
4,seattle,1,Sunday,6,2014,11:59,-122.329379,47.685960,77XX BLOCK OF SUNNYSIDE AV N,J,Green Lake,THEFT-OTH,OTHER PROPERTY


In [154]:
print "SF: %s" % ",".join(sorted(set(sf_normalized["Category"].tolist())))
print "Seattle: %s" % ",".join(sorted(set(seattle_normalized["Category"].tolist())))

print seattle_normalized[seattle_normalized["Category"] == "DISPUTE-CIVIL PROPERTY (AUTO)"].head(10)

category_subset = {
    "RECKLESS BURNING": "VANDALISM",
    "THEFT-OTH": "LARCENY/THEFT",
    "BURGLARY-FORCE-NONRES": "BURGLARY",
    "INJURY - ACCIDENTAL": "OTHER OFFENSES",
    "ANIMAL-BITE": "ASSAULT",
    "ANIMAL-CRUELTY": "DISORDERLY CONDUCT",
    "ANIMAL-OTH": "OTHER OFFENSES",
    "ASSLT-AGG-BODYFORCE": "ASSAULT",
    "ASSLT-AGG-GUN": "ASSAULT",
    "ASSLT-AGG-POLICE-BODYFORCE": "ASSAULT",
    "ASSLT-AGG-POLICE-GUN": "ASSAULT",
    "ASSLT-AGG-POLICE-WEAPON": "ASSAULT",
    "ASSLT-AGG-WEAPON": "ASSAULT",
    "ASSLT-NONAGG": "ASSAULT",
    "ASSLT-NONAGG-POLICE": "ASSAULT",
    "BIAS INCIDENT": "NON-CRIMINAL",
    "BURGLARY-FORCE-RES": "BURGLARY",
    "BURGLARY-NOFORCE-NONRES": "BURGLARY",
    "BURGLARY-NOFORCE-RES": "BURGLARY",
    "BURGLARY-SECURE PARKING-NONRES": "BURGLARY",
    "BURGLARY-SECURE PARKING-RES": "BURGLARY",
    "COUNTERFEIT": "FORGERY/COUNTERFEITING",
    "DISPUTE-CIVIL PROPERTY (AUTO)": "NON-CRIMINAL",
    "DISPUTE-CIVIL PROPERTY (NON AU": "NON-CRIMINAL",
    "DISPUTE-OTH": "NON-CRIMINAL",
    "DISTURBANCE-NOISE": "NON-CRIMINAL",
    "DISTURBANCE-OTH": "NON-CRIMINAL",
    "DRIVE-BY": "ASSAULT",
    "DUI-DRUGS": "DRUG/NARCOTIC",
    "DUI-LIQUOR": "LIQUOR LAWS",
    "ELUDING-FELONY FLIGHT": "RUNAWAY",
    "EMBEZZLE": "EMBEZZLEMENT",
    "ENDANGERMENT": "OTHER OFFENSES",
    "ESCAPE": "RUNAWAY",
    "FALSE REPORT": "OTHER OFFENSES",
    "FIREWORK-POSSESS": "OTHER OFFENSES",
    "FIREWORK-USE": "OTHER OFFENSES",
    "FORGERY-CHECK": "FORGERY/COUNTERFEITING",
    "FORGERY-CREDIT CARD": "FORGERY/COUNTERFEITING",
    "FORGERY-OTH": "FORGERY/COUNTERFEITING",
    "FRAUD-CHECK": "FRAUD",
    "FRAUD-COMPUTER": "FRAUD",
    "FRAUD-CREDIT CARD": "FRAUD",
    "FRAUD-IDENTITY THEFT": "FRAUD",
    "FRAUD-OTHER": "FRAUD",
    "FRAUD-WIRE-ELECTRONIC": "FRAUD"
}
for category, subset in category_subset.iteritems():
    seattle_normalized.loc[seattle_normalized["Category"] == category, "Category"] = subset
#seattle_normalized["Category"] = seattle_normalized["Category"] \
#    .map(lambda x: category_subset[x] if x in category_subset else x)
print "Seattle: %d" % len(set(seattle_normalized["Category"].tolist()))

SF: ARSON,ASSAULT,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,ROBBERY,RUNAWAY,SECONDARY CODES,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Seattle: ,ASSAULT,BURGLARY,DISORDERLY CONDUCT,DRUG/NARCOTIC,EMBEZZLEMENT,FORGERY/COUNTERFEITING,FRAUD,HARASSMENT,HOMICIDE-JUST-GUN,HOMICIDE-JUST-WEAPON,HOMICIDE-PREMEDITATED-GUN,ILLEGAL DUMPING,INJURY - OTHER,LARCENY/THEFT,LIQUOR LAW VIOLATION,LIQUOR LAWS,MALICIOUS HARASSMENT,NARC-DRUG TRAFFIC LOITERING,NARC-EQUIPMENT/PARAPHENALIA,NARC-FORGERY-PRESCRIPTION,NARC-FOUND-AMPHETAMINE,NARC-FOUND-COCAINE,NARC-FOUND-HEROIN,NARC-FOUND-MARIJU,NARC-FOUND-METH,NARC-FOUND-OPIUM,NARC-FOUND-OTHER,NARC-FOUND-SYNTHETIC,NARC-FRAUD-PRESCRIPTION,NARC-POSSESS-AMPHETAMINE,NARC-P